In [68]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [69]:
#extracting the webpage that we want to parse
def extract(page1, page2=None):
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari"}
    url = f"{page1}={page2}"
    if page2 == None:
        url = f"{page1}"
    r = requests.get(url, headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [70]:
#transofmring our raw html data into a readable format
def transform_2(soup, destination):
    div = soup.find_all('div', class_ = 'slider_container')
    for i in div: 
        title = i.find('div', class_= 'heading4 color-text-primary singleLineTitle tapItem-gutter').text.strip()
        name = i.find('span', class_ = "companyName").text.strip()
        location = i.find('div', class_ = "heading6 company_location tapItem-gutter companyInfo").text.strip()
        rating = i.find('span', class_ = "ratingNumber").text.strip()
        summary = i.find('div', class_= 'job-snippet').text.strip()
        
        #loading our new variables into a dictionary
        role = {
            'Title': title,
            'Company_Name': name,
            'Location': location,
            'Rating': rating,
            'Summary': summary
        } 
        destination.append(role)
        
def parse_indeed(company, destination, search1=None, search2=None):
    for i in range(0, 100, 10):
        c = extract(f"https://www.indeed.com/jobs?q={search1}%20{search2}%20{company}&vjk=877ae4112485d7c7&start", 0)
        transform_2(c, destination)


In [75]:
#loading our nationwide data into a pandas df
nationwide = []
parse_indeed('Nationwide', nationwide,'Data', 'Analytics')

In [77]:
#pulling data from wikipedia list of top 100 technology comapnies, for further data collection
r = requests.get("https://en.wikipedia.org/wiki/List_of_largest_Internet_companies")
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[1]
top_100 = df['Company']
top_100.at[1]='Google'
arr = np.array(top_100)
rand_arr = np.random.choice(arr, 3)

array(['Kwai', 'Rackspace', 'Expedia'], dtype=object)

In [62]:
total = []

for x in arr:
    for i in range(0, 100, 10):
        try:
            y = extract(f"https://www.indeed.com/jobs?q=Data%20Analytics%20{x}&vjk=877ae4112485d7c7&start", 0)
            transform_2(y, total)
        except AttributeError:
            print('Ignore')